# Big Data (UBA) -  2024

## Trabajo Práctico 4

### Fecha de entrega:




<font color='red'> Viernes 13 de Diciembre a las 13:00 hs </font>

In [16]:
# Instalacion de paquetes/librerias

import requests
import json
import time
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from bs4 import BeautifulSoup
import locale
import datetime
import seaborn as sns
import plotly.express as px
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.stats import gaussian_kde

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score
from sklearn.metrics import RocCurveDisplay
from sklearn.neighbors import KNeighborsClassifier

from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import RocCurveDisplay
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.neighbors import KNeighborsClassifier

### Parte I: Análisis de la base de hogares y tipo de ocupación

#### 1. Exploren el diseño de registro de la base de hogar: a priori, ¿qué variables creen pueden ser predictivas de la desocupación y seria útil incluir para perfeccionar el ejercicio del TP3? Mencionen estas variables y justifiquen su elección.

Todas las variables que caracterice el tipo de hogar y su construccon pueden ayudar a perfeccionar la estimacion de desocupados (u ocupados) al funcionar como un proxy de status socioeconomico. Especilamente las que estan presente en la construccion de las metrica de pobreza sobre Necesidades Basicas Insatisfechas (baño, numero de habitaciones, condiciones de hacinamiento). Ademas, se podria incorporar las variables de la seccion "¿En los últimos tres meses, las personas de este hogar han vivido...": muchas preguntan indagan directamente por los ingresos del hogar. En caso de no tener nulos, podria dotar al modelo de un significativo poder explicativo. Por ultimo, la cantidad de miembros del hogar y si posee servicio domestico parecen ser otras dos buenas complementaciones.

#### 2.	Descarguen la base de microdatos de la EPH correspondiente al primer trimestre de 2004 y 2024 en formato .dta y .xls, respectivamente. La base de hogares se llama Hogar_t104.dta y usu_hogar_T124.xls, respectivamente. Eliminen todas las observaciones que no corresponden a los aglomerados de Ciuidad Autónoma de Buenos Aires o Gran Buenos Aires y unan ambos trimestres en una sola base. Esto es, a la base de la encuesta individual de cada año (que usaron en el TP3) unan la base de la encuesta de hogar. Asegúrese de estar usando las variables CODUSU y NRO_Hogar para el merge.

In [21]:
# Leer la base de datos para 2004 (archivo .dta)
df_2004 = pd.read_stata('/content/Hogar_t104.dta')

# Leer la base de datos para 2024 (archivo .xls)
df_2024 = pd.read_excel('/content/usu_hogar_T124.xlsx')

# Leer la base de datos del TP3 (archivo .xls)
df_2004_2024_caba_gba_ind = pd.read_excel('/content/df_2004_2024_caba_gba_ind.xlsx')

In [22]:
# Modificamos ciertas cuestiones para poder hacer el merge entre bases.
# Mas especificamente, la adaptamos la base 2004 al formato de la 2024

## Paso las variables de 2004 a mayusculas para que haga el concat de manera correcta

df_2004.columns = df_2004.columns.str.upper()

## Algunas variables de la EPH 2004 tienen datos en STR. Las paso al mismo codigo numerico
## que la EPH 2024

### Cambiar "Si" a 1 y "No" a 0
df_2004.replace({'Si': 1, 'Sí': 1, 'No': 2}, inplace=True)

df_2004_2024_caba_gba_ind.replace({'Si': 1, 'Sí': 1, 'No': 2}, inplace=True)

### Cambiar valores específicos para "Servicio doméstico"
df_2004.replace({'Servicio doméstico': 96,
    'Otra persona que no vive en el hogar': 97,
    'Ninguna': 98
}, inplace=True)


df_2004_2024_caba_gba_ind.replace({'Servicio doméstico': 96,
    'Otra persona que no vive en el hogar': 97,
    'Ninguna': 98
}, inplace=True)

### Diccionarios de reemplazo

### AGLOMERADO
cambios_aglomerado = {
    "Gran La Plata": 2, "Bahía Blanca - Cerri": 3, "Gran Rosario": 4, "Gran Santa Fé": 5,
    "Gran Paraná": 6, "Posadas": 7, "Gran Resistencia": 8, "Cdro. Rivadavia – Rada Tilly": 9,
    "Gran Mendoza": 10, "Corrientes": 12, "Gran Córdoba": 13, "Concordia": 14,
    "Formosa": 15, "Neuquén – Plottier": 17, "S.del Estero - La Banda": 18,
    "Jujuy - Palpalá": 19, "Río Gallegos": 20, "Gran Catamarca": 22, "Salta": 23,
    "La Rioja": 25, "San Luis - El Chorrillo": 26, "Gran San Juan": 27,
    "Gran Tucumán - T. Viejo": 29, "Santa Rosa - Toay": 30, "Ushuaia - Río Grande": 31,
    "Ciudad de Buenos Aires": 32, "Partidos del GBA": 33, "Mar del Plata - Batán": 34,
    "Río Cuarto": 36
}

### IV1
cambios_iv1 = {
    "Casa": 1, "Departamento": 2, "Pieza de inquilinato": 3,
    "Pieza en hotel / pensión": 4, "Local no construido para habitación": 5, "Otros": 6
}

### IV3
cambios_iv3 = {
    "Mosaico / baldosa / madera / cerámica / alfombra": 1, "Cemento / ladrillo fijo": 2,
    "Ladrillo suelto / tierra": 3, "Otro": 4
}

### IV4
cambios_iv4 = {
    "Membrana / cubierta asfáltica": 1, "Baldosa / losa sin cubierta": 2, "Pizarra / teja": 3,
    "Chapa de metal sin cubierta": 4, "Chapa de fibrocemento / plástico": 5,
    "Chapa de cartón": 6, "Caña / tabla / paja con barro / paja sola": 7,
    "N/S. Depto en propiedad horizontal": 9
}

### IV6
cambios_iv6 = {
    "Por cañería dentro de la vivienda": 1,
    "Fuera de la vivienda pero dentro del terreno": 2,
    "Fuera del terreno": 3
}

### IV7
cambios_iv7 = {
    "Red pública (agua corriente)": 1, "Perforación con bomba a motor": 2,
    "Perforación con bomba manual": 3, "Otra fuente": 4
}

### IV9
cambios_iv9 = {
    "Dentro de la vivienda": 1,
    "Fuera de la vivienda pero dentro del terreno": 2,
    "Fuera del terreno": 3
}

### IV10
cambios_iv10 = {
    "Inodoro con botón / mochila / cadena y arrastre de agua": 1,
    "Inodoro sin botón / cadena y con arrastre de agua (a balde)": 2,
    "Letrina (sin arrastre de agua)": 3
}

### IV11
cambios_iv11 = {
    "A red pública (cloaca)": 1, "A cámara séptica y pozo ciego": 2,
    "Sólo a pozo ciego": 3, "A hoyo / excavación en la tierra": 4
}

### II7
cambios_ii7 = {
    "Propietario de la vivienda y el terreno": 1, "Propietario de la vivienda solamente": 2,
    "Inquilino / arrendatario de la vivienda": 3, "Ocupante por pago de impuestos / expensas": 4,
    "Ocupante en relación de dependencia": 5, "Ocupante gratuito (con permiso)": 6,
    "Ocupante de hecho (sin permiso)": 7, "Está en sucesión": 8, "Otra situación": 9
}

### II8
cambios_ii8 = {
    "Gas de red": 1, "Gas de tubo / garrafa": 2,
    "Kerosene / leña/ carbón": 3, "Otro": 4
}

### II9
cambios_ii9 = {
    "Uso exclusivo del hogar": 1,
    "Compartido con otro/s hogar/es de la misma vivienda": 2,
    "Compartido con otra/s vivienda/s": 3,
    "No tiene baño": 4
}

df_2004['AGLOMERADO'].replace(cambios_aglomerado, inplace=True)
df_2004['IV1'].replace(cambios_iv1, inplace=True)
df_2004['IV3'].replace(cambios_iv3, inplace=True)
df_2004['IV4'].replace(cambios_iv4, inplace=True)
df_2004['IV6'].replace(cambios_iv6, inplace=True)
df_2004['IV7'].replace(cambios_iv7, inplace=True)
df_2004['IV9'].replace(cambios_iv9, inplace=True)
df_2004['IV10'].replace(cambios_iv10, inplace=True)
df_2004['IV11'].replace(cambios_iv11, inplace=True)
df_2004['II7'].replace(cambios_ii7, inplace=True)
df_2004['II8'].replace(cambios_ii8, inplace=True)
df_2004['II9'].replace(cambios_ii9, inplace=True)

# Filtrar las observaciones para CABA (AGLOMERADO = 32) y GBA (AGLOMERADO = 33) en 2004 y 2024

df_2004_caba_gba = df_2004[df_2004['AGLOMERADO'].isin([32,33])]

df_2024_caba_gba = df_2024[df_2024['AGLOMERADO'].isin([32,33])]

# Unir ambos trimestres de la EPH Hogar

df_2004_2024_caba_gba_hog = pd.concat([df_2004_caba_gba, df_2024_caba_gba], ignore_index=True)

# Unir la EPH Hogar con la EPH Individuo, para iguales trimestres de
# 2004 y 2024, respectivamente.

df_2004_2024_caba_gba = pd.merge(df_2004_2024_caba_gba_ind, df_2004_2024_caba_gba_hog, on=["CODUSU", "NRO_HOGAR"
], how="inner")

df_2004_2024_caba_gba

<ipython-input-22-48fe843600be>:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_2004.replace({'Si': 1, 'Sí': 1, 'No': 2}, inplace=True)
<ipython-input-22-48fe843600be>:12: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_2004.replace({'Si': 1, 'Sí': 1, 'No': 2}, inplace=True)
<ipython-input-22-48fe843600be>:12: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_optio

,CODUSU,NRO_HOGAR,COMPONENTE,H15,ANO4_x,TRIMESTRE_x,REGION_x,MAS_500_x,AGLOMERADO_x,PONDERA_x,...,PDECCFR_y,ADECCFR_y,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4,IDIMPH,PONDIH_y
0,125098,1,1,1,2004,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1299,...,,04,1.0,0.0,98.0,0.0,0.0,0.0,00000,NaN
1,125397,1,1,1,2004,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,2018,...,,00,1.0,0.0,98.0,0.0,0.0,0.0,00000,NaN
2,125515,1,1,1,2004,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1486,...,,05,1.0,0.0,98.0,0.0,0.0,0.0,00010,NaN
3,125558,1,1,1,2004,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1674,...,,06,2.0,0.0,3.0,4.0,0.0,0.0,00000,NaN
4,125558,1,2,1,2004,1er. Trimestre,Gran Buenos Aires,S,Ciudad de Buenos Aires,1674,...,,06,2.0,0.0,3.0,4.0,0.0,0.0,00000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14693,TQRMNOPUQHKMKNCDEIJAH00861661,1,1,1,2024,1,1,S,33,3410,...,NaN,12,99,0.0,99.0,0.0,0.0,0.0,NaN,0.0
14694,TQRMNOPUQHKMKNCDEIJAH00861661,1,2,2,2024,1,1,S,33,3410,...,NaN,12,99,0.0,99.0,0.0,0.0,0.0,NaN,0.0
14695,TQRMNOPUQHKMKNCDEIJAH00861661,1,3,2,2024,1,1,S,33,3410,...,NaN,12,99,0.0,99.0,0.0,0.0,0.0,NaN,0.0
14696,TQRMNOPUQHKMKNCDEIJAH00861661,1,4,2,2024,1,1,S,33,3410,...,NaN,12,99,0.0,99.0,0.0,0.0,0.0,NaN,0.0


#### 3.	Limpien la base de datos tomando criterios que hagan sentido. Explicar cualquier decisión como el tratamiento de valores faltantes (missing values), extremos (outliers), o variables categóricas. Justifique sus decisiones.

In [28]:
# La primera limpieza necesaria es eliminar las columnas que terminana en "_x" y
# dejar las de "_y". Ambas eran variables iguales y me quedo con las de hogar porque son de
# formato float64.

# Eliminar todas las columnas que terminan en '_x'
df_2004_2024_caba_gba_fil = df_2004_2024_caba_gba[[col for col in df_2004_2024_caba_gba.columns if not col.endswith('_x')]]

# Renombrar las columnas que terminan en '_y' para quitar el sufijo
df_2004_2024_caba_gba_fil.columns = [col[:-2] if col.endswith('_y') else col for col in df_2004_2024_caba_gba_fil.columns]

# Limpiar la cantidad de ambientes mayores a 11
# Eliminar la cantidad de miembros del hogar mayores a 10
# Edad mayor a cero

# Eliminar filas donde 'IV2' > 11
df_2004_2024_caba_gba_fil = df_2004_2024_caba_gba_fil[df_2004_2024_caba_gba_fil['IV2'] <= 11]

# Eliminar filas donde 'IX_TOT' > 10
df_2004_2024_caba_gba_fil = df_2004_2024_caba_gba_fil[df_2004_2024_caba_gba_fil['IX_TOT'] <= 10]

# Eliminar filas donde 'CH06' <= 0
df_2004_2024_caba_gba_fil = df_2004_2024_caba_gba_fil[df_2004_2024_caba_gba_fil['CH06'] > 0]



In [20]:
df_2004_2024_caba_gba_fil.to_excel("df_2004_2024_caba_gba_fil_v2.xlsx", index=False)

#### 4.	Construya variables (mínimo 3) que no estén en la base pero que sean relevantes para predecir individuos desocupados (por ejemplo, la proporción de personas que trabajan en el hogar).

In [29]:
# Adultos mayores en el hogar

## Filtrar los miembros mayores a 65 años
df_mayores_65 = df_2004_2024_caba_gba_fil[df_2004_2024_caba_gba_fil['CH06'] > 65]

## Agrupar por 'CODUSU' y 'NRO_HOGAR' y contar la cantidad de personas mayores a 65
df_mayores_65_count = df_mayores_65.groupby(['CODUSU', 'NRO_HOGAR']).size().reset_index(name='MIEMBROS_MAYORES_65')

## Unir el conteo al DataFrame original (para agregar la nueva columna)
df_2004_2024_caba_gba_fil = pd.merge(df_2004_2024_caba_gba_fil, df_mayores_65_count, on=['CODUSU', 'NRO_HOGAR'], how='left')

## Rellenar los valores NaN con 0 y convertir la columna a tipo int
df_2004_2024_caba_gba_fil['MIEMBROS_MAYORES_65'] = df_2004_2024_caba_gba_fil['MIEMBROS_MAYORES_65'].fillna(0).astype(int)

# Dummy si el hogar recibe alguna transferencia del estado

df_2004_2024_caba_gba_fil['AYUDA_GOBIERNO'] = (
    (df_2004_2024_caba_gba_fil[['V1', 'V2', 'V21', 'V22', 'V5']] == 1).any(axis=1).astype(int)
)

# Niños en primera infancia en el hogar

df_menores_6 = df_2004_2024_caba_gba_fil[df_2004_2024_caba_gba_fil['CH06'] < 6]
df_menores_6_count = df_menores_6.groupby(['CODUSU', 'NRO_HOGAR']).size().reset_index(name='MIEMBROS_MENORES_6')
df_2004_2024_caba_gba_fil = pd.merge(df_2004_2024_caba_gba_fil, df_menores_6_count, on=['CODUSU', 'NRO_HOGAR'], how='left')
df_2004_2024_caba_gba_fil['MIEMBROS_MENORES_6'] = df_2004_2024_caba_gba_fil['MIEMBROS_MENORES_6'].fillna(0).astype(int)
